<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Smart_Trading_Bot_IB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
SmartZone Trading Bot - Interactive Brokers Edition
Versão: 1.0 (Paper Trading)
Plataforma: Google Colab + IB API

FUNCIONALIDADES:
- Detecção automática de Order Blocks e Fair Value Gaps
- Integração com Interactive Brokers (Paper Trading)
- Gestão de risco dinâmica
- Logs detalhados e visualização em tempo real
"""

#==============================================================================
# PARTE 1: INSTALAÇÃO E IMPORTS
#==============================================================================

# Instalar bibliotecas necessárias
!pip install ib_insync pandas numpy ta-lib python-telegram-bot plotly -q

import asyncio
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from ib_insync import *
import time
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Configuração para async no Colab
import nest_asyncio
nest_asyncio.apply()

#==============================================================================
# PARTE 2: CONFIGURAÇÕES DO BOT
#==============================================================================

class BotConfig:
    """Configurações principais do bot"""

    # CONEXÃO IB
    IB_HOST = '127.0.0.1'  # Localhost (via ngrok se necessário)
    IB_PORT = 7497         # 7497 = Paper Trading, 7496 = Live
    CLIENT_ID = 1

    # SÍMBOLOS
    SYMBOLS = ['BTC', 'ETH']  # Crypto via IB
    EXCHANGE = 'PAXOS'        # Exchange para crypto
    CURRENCY = 'USD'

    # TAMANHO DE POSIÇÃO
    LOT_SIZE = 0.01           # Tamanho pequeno para começar
    MAX_POSITIONS = 2         # Máximo de posições simultâneas

    # GESTÃO DE RISCO
    RISK_REWARD_RATIO = 2.0   # 1:2
    SL_MIN_PIPS = 50
    SL_MAX_PIPS = 250
    SL_BUFFER_PIPS = 20
    TRAILING_START_PIPS = 120
    TRAILING_STEP_PIPS = 40

    # SPREAD E VALIDAÇÃO
    MAX_SPREAD_PIPS = 100
    COOLDOWN_SECONDS = 120    # 2 min entre trades

    # DETECÇÃO SMC
    BARS_ANALYSIS = 50
    OB_MIN_BODY_RATIO = 0.4
    FVG_MIN_PIPS = 30
    ZONE_PROXIMITY = 50       # Pips de proximidade para entrada

    # TIMEFRAME
    TIMEFRAME = '1 hour'      # Barras de 1 hora

    # MODO
    PAPER_TRADING = True      # ✅ SEMPRE True para começar!
    DEBUG = True              # Logs detalhados

    # NOTIFICAÇÕES (opcional)
    TELEGRAM_TOKEN = None     # Adicionar se quiser notificações
    TELEGRAM_CHAT_ID = None

#==============================================================================
# PARTE 3: CLASSE PRINCIPAL DO BOT
#==============================================================================

class SmartZoneBot:
    """Bot de trading com estratégia Smart Money Concepts"""

    def __init__(self, config: BotConfig):
        self.config = config
        self.ib = IB()
        self.active_trades = {}
        self.last_trade_time = {}
        self.ob_bull_zones = []
        self.ob_bear_zones = []
        self.fvg_bull_zones = []
        self.fvg_bear_zones = []
        self.market_bias = "NEUTRO"

        # Estatísticas
        self.stats = {
            'total_trades': 0,
            'wins': 0,
            'losses': 0,
            'total_pnl': 0.0
        }

    async def connect(self):
        """Conecta ao Interactive Brokers"""
        try:
            await self.ib.connectAsync(
                self.config.IB_HOST,
                self.config.IB_PORT,
                clientId=self.config.CLIENT_ID
            )

            mode = "PAPER TRADING ✅" if self.config.PAPER_TRADING else "LIVE ⚠️"
            print(f"✓ Conectado ao IB ({mode})")
            print(f"✓ Conta: {self.ib.wrapper.accounts}")

            return True

        except Exception as e:
            print(f"✗ Erro ao conectar: {e}")
            return False

    def create_contract(self, symbol: str):
        """Cria contrato para crypto"""
        contract = Crypto(symbol, self.config.EXCHANGE, self.config.CURRENCY)
        return contract

    async def get_historical_data(self, symbol: str, bars: int = 100):
        """Obtém dados históricos"""
        contract = self.create_contract(symbol)

        # Qualifica o contrato
        await self.ib.qualifyContractsAsync(contract)

        # Solicita dados históricos
        bars_data = await self.ib.reqHistoricalDataAsync(
            contract,
            endDateTime='',
            durationStr=f'{bars} H',
            barSizeSetting='1 hour',
            whatToShow='MIDPOINT',
            useRTH=False
        )

        # Converte para DataFrame
        df = util.df(bars_data)
        df.set_index('date', inplace=True)

        return df

    def detect_order_blocks(self, df: pd.DataFrame):
        """Detecta Order Blocks (adaptado do MQL4)"""
        self.ob_bull_zones = []
        self.ob_bear_zones = []

        for i in range(2, min(len(df), self.config.BARS_ANALYSIS)):
            row = df.iloc[-i]

            body = abs(row['close'] - row['open'])
            range_val = row['high'] - row['low']

            if range_val == 0:
                continue

            body_ratio = body / range_val

            # OB Bullish (vela baixista seguida de alta)
            if row['close'] < row['open'] and body_ratio >= self.config.OB_MIN_BODY_RATIO:
                # Verifica confirmação
                if i >= 2:
                    next_row = df.iloc[-i+1]
                    if next_row['close'] > row['high']:
                        self.ob_bull_zones.append({
                            'high': row['high'],
                            'low': row['low'],
                            'time': row.name
                        })

            # OB Bearish (vela altista seguida de baixa)
            if row['close'] > row['open'] and body_ratio >= self.config.OB_MIN_BODY_RATIO:
                if i >= 2:
                    next_row = df.iloc[-i+1]
                    if next_row['close'] < row['low']:
                        self.ob_bear_zones.append({
                            'high': row['high'],
                            'low': row['low'],
                            'time': row.name
                        })

        # Limita a 5 zonas
        self.ob_bull_zones = self.ob_bull_zones[:5]
        self.ob_bear_zones = self.ob_bear_zones[:5]

    def detect_fvg(self, df: pd.DataFrame):
        """Detecta Fair Value Gaps"""
        self.fvg_bull_zones = []
        self.fvg_bear_zones = []

        for i in range(2, min(len(df), self.config.BARS_ANALYSIS)):
            if i + 1 >= len(df):
                break

            current = df.iloc[-i-1]
            middle = df.iloc[-i]
            previous = df.iloc[-i+1]

            # FVG Bullish
            gap = middle['low'] - previous['high']
            if gap > self.config.FVG_MIN_PIPS and middle['close'] > previous['close']:
                self.fvg_bull_zones.append({
                    'high': middle['low'],
                    'low': previous['high'],
                    'time': middle.name
                })

            # FVG Bearish
            gap = previous['low'] - middle['high']
            if gap > self.config.FVG_MIN_PIPS and middle['close'] < previous['close']:
                self.fvg_bear_zones.append({
                    'high': previous['low'],
                    'low': middle['high'],
                    'time': middle.name
                })

        self.fvg_bull_zones = self.fvg_bull_zones[:5]
        self.fvg_bear_zones = self.fvg_bear_zones[:5]

    def analyze_market_bias(self, df: pd.DataFrame):
        """Analisa viés do mercado"""
        buy_points = len(self.ob_bull_zones) * 2 + len(self.fvg_bull_zones)
        sell_points = len(self.ob_bear_zones) * 2 + len(self.fvg_bear_zones)

        if buy_points > sell_points:
            self.market_bias = "BUY"
        elif sell_points > buy_points:
            self.market_bias = "SELL"
        else:
            self.market_bias = "NEUTRO"

    def check_spread(self, ticker):
        """Verifica se o spread está aceitável"""
        if ticker.bid and ticker.ask:
            spread = ticker.ask - ticker.bid
            spread_pips = spread  # Para crypto, considerar em USD

            if spread_pips > self.config.MAX_SPREAD_PIPS:
                if self.config.DEBUG:
                    print(f"⚠ Spread alto: {spread_pips:.2f}")
                return False

            return True
        return False

    async def check_entry_signal(self, symbol: str, df: pd.DataFrame):
        """Verifica sinais de entrada"""

        # Cooldown
        last_trade = self.last_trade_time.get(symbol, 0)
        if time.time() - last_trade < self.config.COOLDOWN_SECONDS:
            return None

        # Preço atual
        contract = self.create_contract(symbol)
        ticker = self.ib.reqMktData(contract)
        await asyncio.sleep(2)  # Aguarda dados

        if not ticker.last or ticker.last == 0:
            return None

        current_price = ticker.last

        # Verifica spread
        if not self.check_spread(ticker):
            return None

        proximity = self.config.ZONE_PROXIMITY

        # Verifica zonas de compra
        if self.market_bias in ["BUY", "NEUTRO"]:
            for zone in self.ob_bull_zones:
                if zone['low'] - proximity <= current_price <= zone['high'] + proximity:
                    return {
                        'type': 'BUY',
                        'reason': f'OB Bull Zone',
                        'zone_high': zone['high'],
                        'zone_low': zone['low'],
                        'entry_price': current_price
                    }

            for zone in self.fvg_bull_zones:
                if zone['low'] - proximity <= current_price <= zone['high'] + proximity:
                    return {
                        'type': 'BUY',
                        'reason': f'FVG Bull Zone',
                        'zone_high': zone['high'],
                        'zone_low': zone['low'],
                        'entry_price': current_price
                    }

        # Verifica zonas de venda
        if self.market_bias in ["SELL", "NEUTRO"]:
            for zone in self.ob_bear_zones:
                if zone['low'] - proximity <= current_price <= zone['high'] + proximity:
                    return {
                        'type': 'SELL',
                        'reason': f'OB Bear Zone',
                        'zone_high': zone['high'],
                        'zone_low': zone['low'],
                        'entry_price': current_price
                    }

            for zone in self.fvg_bear_zones:
                if zone['low'] - proximity <= current_price <= zone['high'] + proximity:
                    return {
                        'type': 'SELL',
                        'reason': f'FVG Bear Zone',
                        'zone_high': zone['high'],
                        'zone_low': zone['low'],
                        'entry_price': current_price
                    }

        return None

    def calculate_sl_tp(self, signal: dict):
        """Calcula Stop Loss e Take Profit"""
        entry = signal['entry_price']

        if signal['type'] == 'BUY':
            # SL dinâmico baseado na zona
            sl = signal['zone_low'] - self.config.SL_BUFFER_PIPS
            sl_distance = entry - sl

            # Limites
            if sl_distance > self.config.SL_MAX_PIPS:
                sl = entry - self.config.SL_MAX_PIPS
            elif sl_distance < self.config.SL_MIN_PIPS:
                sl = entry - self.config.SL_MIN_PIPS

            # TP com Risk:Reward
            tp = entry + (entry - sl) * self.config.RISK_REWARD_RATIO

        else:  # SELL
            sl = signal['zone_high'] + self.config.SL_BUFFER_PIPS
            sl_distance = sl - entry

            if sl_distance > self.config.SL_MAX_PIPS:
                sl = entry + self.config.SL_MAX_PIPS
            elif sl_distance < self.config.SL_MIN_PIPS:
                sl = entry + self.config.SL_MIN_PIPS

            tp = entry - (sl - entry) * self.config.RISK_REWARD_RATIO

        return round(sl, 2), round(tp, 2)

    async def open_trade(self, symbol: str, signal: dict):
        """Abre uma ordem"""

        # Limita número de posições
        if len(self.active_trades) >= self.config.MAX_POSITIONS:
            print(f"⚠ Máximo de posições atingido ({self.config.MAX_POSITIONS})")
            return

        contract = self.create_contract(symbol)

        # Calcula SL e TP
        sl, tp = self.calculate_sl_tp(signal)

        # Cria ordem
        action = signal['type']
        order = MarketOrder(
            action=action,
            totalQuantity=self.config.LOT_SIZE,
            tif='GTC'
        )

        # Adiciona ordens de SL e TP (bracket)
        parent_order = order
        parent_order.orderId = self.ib.client.getReqId()
        parent_order.transmit = False

        # Stop Loss
        sl_order = StopOrder(
            action='SELL' if action == 'BUY' else 'BUY',
            totalQuantity=self.config.LOT_SIZE,
            stopPrice=sl,
            parentId=parent_order.orderId,
            transmit=False
        )

        # Take Profit
        tp_order = LimitOrder(
            action='SELL' if action == 'BUY' else 'BUY',
            totalQuantity=self.config.LOT_SIZE,
            lmtPrice=tp,
            parentId=parent_order.orderId,
            transmit=True  # Transmit all orders
        )

        try:
            # Envia ordens em bracket
            parent_trade = self.ib.placeOrder(contract, parent_order)
            sl_trade = self.ib.placeOrder(contract, sl_order)
            tp_trade = self.ib.placeOrder(contract, tp_order)

            # Aguarda confirmação
            await asyncio.sleep(2)

            print(f"\n{'='*60}")
            print(f"✓ TRADE ABERTO: {symbol} {action}")
            print(f"Motivo: {signal['reason']}")
            print(f"Entrada: ${signal['entry_price']:.2f}")
            print(f"Stop Loss: ${sl:.2f}")
            print(f"Take Profit: ${tp:.2f}")
            print(f"R:R: 1:{self.config.RISK_REWARD_RATIO}")
            print(f"{'='*60}\n")

            # Salva trade ativo
            self.active_trades[symbol] = {
                'parent': parent_trade,
                'sl': sl_trade,
                'tp': tp_trade,
                'signal': signal,
                'sl_price': sl,
                'tp_price': tp,
                'entry_time': datetime.now()
            }

            self.last_trade_time[symbol] = time.time()
            self.stats['total_trades'] += 1

        except Exception as e:
            print(f"✗ Erro ao abrir trade: {e}")

    async def manage_trades(self):
        """Gerencia trades ativos (trailing stop, etc.)"""
        for symbol, trade_info in list(self.active_trades.items()):

            contract = self.create_contract(symbol)
            ticker = self.ib.reqMktData(contract)
            await asyncio.sleep(1)

            if not ticker.last:
                continue

            current_price = ticker.last
            entry_price = trade_info['signal']['entry_price']

            # Calcula lucro em pips
            if trade_info['signal']['type'] == 'BUY':
                profit_pips = current_price - entry_price
            else:
                profit_pips = entry_price - current_price

            # Trailing Stop
            if profit_pips >= self.config.TRAILING_START_PIPS:
                # Implementar trailing stop aqui
                if self.config.DEBUG:
                    print(f"✓ {symbol}: Lucro {profit_pips:.2f} pips (trailing ativo)")

            # Verifica se ordem foi fechada
            if trade_info['parent'].orderStatus.status in ['Filled', 'Cancelled']:
                pnl = ticker.last - entry_price if trade_info['signal']['type'] == 'BUY' else entry_price - ticker.last

                if pnl > 0:
                    self.stats['wins'] += 1
                    print(f"✓ {symbol}: GANHO ${pnl:.2f}")
                else:
                    self.stats['losses'] += 1
                    print(f"✗ {symbol}: PERDA ${pnl:.2f}")

                self.stats['total_pnl'] += pnl
                del self.active_trades[symbol]

    def display_status(self, symbol: str, df: pd.DataFrame):
        """Exibe status do bot"""
        clear_output(wait=True)

        print("╔" + "="*78 + "╗")
        print("║" + " "*20 + "SMARTZONE BOT - INTERACTIVE BROKERS" + " "*23 + "║")
        print("╚" + "="*78 + "╝\n")

        # Status da conexão
        mode = "📄 PAPER TRADING" if self.config.PAPER_TRADING else "⚠️ LIVE TRADING"
        print(f"Status: {'🟢 Conectado' if self.ib.isConnected() else '🔴 Desconectado'} | {mode}")
        print(f"Símbolo: {symbol} | Viés: {self.market_bias} | Hora: {datetime.now().strftime('%H:%M:%S')}")
        print(f"\n{'─'*80}\n")

        # Preço atual
        if not df.empty:
            last_price = df.iloc[-1]['close']
            print(f"💰 Preço Atual: ${last_price:.2f}")

        print(f"\n{'─'*80}\n")

        # Zonas detectadas
        print(f"📊 ZONAS DETECTADAS:")
        print(f"   • Order Blocks Bull: {len(self.ob_bull_zones)}")
        print(f"   • Order Blocks Bear: {len(self.ob_bear_zones)}")
        print(f"   • FVG Bull: {len(self.fvg_bull_zones)}")
        print(f"   • FVG Bear: {len(self.fvg_bear_zones)}")

        print(f"\n{'─'*80}\n")

        # Trades ativos
        print(f"📈 TRADES ATIVOS: {len(self.active_trades)}/{self.config.MAX_POSITIONS}")
        for sym, trade in self.active_trades.items():
            duration = datetime.now() - trade['entry_time']
            print(f"   • {sym} {trade['signal']['type']} | ${trade['signal']['entry_price']:.2f} | {duration}")

        print(f"\n{'─'*80}\n")

        # Estatísticas
        win_rate = (self.stats['wins'] / self.stats['total_trades'] * 100) if self.stats['total_trades'] > 0 else 0
        print(f"📊 ESTATÍSTICAS:")
        print(f"   • Total Trades: {self.stats['total_trades']}")
        print(f"   • Wins: {self.stats['wins']} | Losses: {self.stats['losses']}")
        print(f"   • Win Rate: {win_rate:.1f}%")
        print(f"   • PnL Total: ${self.stats['total_pnl']:.2f}")

        print(f"\n{'─'*80}\n")

    async def run(self):
        """Loop principal do bot"""

        if not await self.connect():
            print("✗ Falha na conexão. Verifique se o TWS/Gateway está rodando.")
            return

        print("\n🚀 BOT INICIADO!\n")

        iteration = 0

        try:
            while True:
                iteration += 1

                for symbol in self.config.SYMBOLS:

                    # Obtém dados históricos
                    df = await self.get_historical_data(symbol)

                    if df.empty:
                        continue

                    # Análise de mercado
                    self.detect_order_blocks(df)
                    self.detect_fvg(df)
                    self.analyze_market_bias(df)

                    # Exibe status
                    self.display_status(symbol, df)

                    # Verifica sinais de entrada
                    if len(self.active_trades) < self.config.MAX_POSITIONS:
                        signal = await self.check_entry_signal(symbol, df)

                        if signal:
                            await self.open_trade(symbol, signal)

                    # Gerencia trades ativos
                    await self.manage_trades()

                # Aguarda próxima iteração (5 minutos)
                print(f"\n⏳ Próxima análise em 5 minutos... (Iteração #{iteration})")
                await asyncio.sleep(300)

        except KeyboardInterrupt:
            print("\n\n⚠ Bot interrompido pelo usuário")
        except Exception as e:
            print(f"\n\n✗ Erro: {e}")
        finally:
            self.ib.disconnect()
            print("✓ Desconectado do IB")

#==============================================================================
# PARTE 4: EXECUÇÃO
#==============================================================================

async def main():
    """Função principal"""

    print("""
    ╔════════════════════════════════════════════════════════════════════╗
    ║                                                                    ║
    ║              SMARTZONE BOT - INTERACTIVE BROKERS                   ║
    ║                      PAPER TRADING MODE                            ║
    ║                                                                    ║
    ╚════════════════════════════════════════════════════════════════════╝

    ⚠️  IMPORTANTE:
    1. Certifique-se de que o TWS/IB Gateway está rodando
    2. Configure para PAPER TRADING (porta 7497)
    3. Habilite API em TWS: File > Global Configuration > API > Settings
    4. Aceite conexões da API quando solicitado

    ✅ Pressione Ctrl+C para parar o bot a qualquer momento
    """)

    # Cria instância do bot
    config = BotConfig()
    bot = SmartZoneBot(config)

    # Inicia o bot
    await bot.run()

# Executa o bot
if __name__ == "__main__":
    asyncio.run(main())

ERROR:ib_insync.client:API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 7497)")
ERROR:ib_insync.client:Make sure API port on TWS/IBG is open



    ╔════════════════════════════════════════════════════════════════════╗
    ║                                                                    ║
    ║              SMARTZONE BOT - INTERACTIVE BROKERS                   ║
    ║                      PAPER TRADING MODE                            ║
    ║                                                                    ║
    ╚════════════════════════════════════════════════════════════════════╝
    
    ⚠️  IMPORTANTE:
    1. Certifique-se de que o TWS/IB Gateway está rodando
    2. Configure para PAPER TRADING (porta 7497)
    3. Habilite API em TWS: File > Global Configuration > API > Settings
    4. Aceite conexões da API quando solicitado
    
    ✅ Pressione Ctrl+C para parar o bot a qualquer momento
    
✗ Erro ao conectar: [Errno 111] Connect call failed ('127.0.0.1', 7497)
✗ Falha na conexão. Verifique se o TWS/Gateway está rodando.
